In [2]:
pip install PyWavelets


   ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/4.2 MB 5.6 MB/s eta 0:00:01
   ----------------- ---------------------- 1.8/4.2 MB 5.9 MB/s eta 0:00:01
   ------------------------ --------------- 2.6/4.2 MB 5.6 MB/s eta 0:00:01
   ------------------------------------- -- 3.9/4.2 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------- 4.2/4.2 MB 5.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import numpy as np
import pywt
import matplotlib.pyplot as plt

# --- Parameters ---
WAVELET = 'haar'  # Simple and fast wavelet. Try 'db1', 'sym2' for alternatives.
LEVEL = 1         # Single-level decomposition
THRESHOLD_VALUE = 20 # Threshold for high-frequency coefficients (controls compression/loss)
IMAGE_FILE = 'person.jpg' # Replace with your image file name

# --- 1. Load the Image ---
try:
    # Read the image in grayscale
    original_img = cv2.imread(IMAGE_FILE, cv2.IMREAD_GRAYSCALE)
    if original_img is None:
        raise FileNotFoundError
except FileNotFoundError:
    print(f"Error: Image '{IMAGE_FILE}' not found. Creating a dummy 128x128 image.")
    original_img = np.random.randint(0, 256, (128, 128), dtype=np.uint8)

original_img = original_img.astype(np.float32)

# --- 2. Apply Discrete Wavelet Transform (DWT) ---
# Decompose the image into 4 subbands
# LL, (LH, HL, HH)
coeffs = pywt.dwt2(original_img, WAVELET)
LL, (LH, HL, HH) = coeffs

# Note: For multi-level (recursive) decomposition, pywt.wavedec2 is used.

# --- 3. Quantization / Thresholding ---

# Apply hard thresholding to the detail subbands (LH, HL, HH)
# Coefficients with magnitude less than THRESHOLD_VALUE are set to zero.
def apply_thresholding(band, threshold):
    """Sets coefficients below the threshold magnitude to zero."""
    band_out = band.copy()
    band_out[np.abs(band_out) < threshold] = 0.0
    return band_out

LH_t = apply_thresholding(LH, THRESHOLD_VALUE)
HL_t = apply_thresholding(HL, THRESHOLD_VALUE)
HH_t = apply_thresholding(HH, THRESHOLD_VALUE)

# The new set of coefficients for reconstruction
compressed_coeffs = (LL, (LH_t, HL_t, HH_t))

# --- 4 & 5. Subband Encoding and Storing (Approximation) ---
# In a full implementation, the non-zero coefficients (LL, LH_t, HL_t, HH_t)
# would be encoded using RLE/Huffman/Arithmetic coding and stored.

# Approximation of Compression Ratio:
# CR calculation requires actual bitstream size, which is complex.
# We'll use the ratio of non-zero coefficients as a proxy for compression efficiency.

total_coeffs = original_img.size
non_zero_orig = np.sum(LH!=0) + np.sum(HL!=0) + np.sum(HH!=0) + np.sum(LL!=0)
non_zero_comp = np.sum(LH_t!=0) + np.sum(HL_t!=0) + np.sum(HH_t!=0) + np.sum(LL!=0)
# Note: LL is not thresholded, so its non-zero count remains constant.

# --- 6. Reconstruction (Decoding) ---

# Reconstruct the image using the Inverse DWT (IDWT)
reconstructed_img = pywt.idwt2(compressed_coeffs, WAVELET)

# Convert back to 8-bit integer format for display/comparison
reconstructed_img = np.clip(reconstructed_img, 0, 255).astype(np.uint8)
original_img = original_img.astype(np.uint8)

# --- 7. Evaluate Compression Performance ---

# Calculate PSNR (Peak Signal-to-Noise Ratio)
def calculate_psnr(original, reconstructed):
    """Calculates PSNR (in dB) between two images."""
    mse = np.mean((original.astype(np.float32) - reconstructed.astype(np.float32)) ** 2)
    if mse == 0:
        return 100 # Perfect match
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

psnr_value = calculate_psnr(original_img, reconstructed_img)

# Proxy Compression Ratio (based on zeroed coefficients)
coeff_reduction = non_zero_orig / non_zero_comp
# This is a highly simplified proxy, not a true CR.
# A higher value means more coefficients were zeroed out.

print("\n--- Compression Metrics ---")
print(f"Wavelet Used: {WAVELET}")
print(f"Threshold Value: {THRESHOLD_VALUE}")
print(f"Zeroed Coefficients Ratio (Proxy CR): {coeff_reduction:.2f}x")
print(f"PSNR (Quality): {psnr_value:.2f} dB")
print("---------------------------\n")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Original Image
axes[0].imshow(original_img, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

# Reconstructed Image
axes[1].imshow(reconstructed_img, cmap='gray')
axes[1].set_title(f'Reconstructed Image (PSNR: {psnr_value:.2f} dB)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# Visualize Subbands (LL and the thresholded detail bands)
fig, axes = plt.subplots(2, 2, figsize=(8, 8))
titles = ['LL (Approximation)', 'LH (Horizontal)', 'HL (Vertical)', 'HH (Diagonal)']
bands = [LL, LH_t, HL_t, HH_t]

# We use a normalized log scale for visualization to see the details better
def visualize_band(ax, band, title):
    ax.imshow(np.log1p(np.abs(band)), cmap='gray')
    ax.set_title(title)
    ax.axis('off')

visualize_band(axes[0, 0], bands[0], titles[0])
visualize_band(axes[0, 1], bands[1], titles[1])
visualize_band(axes[1, 0], bands[2], titles[2])
visualize_band(axes[1, 1], bands[3], titles[3])

plt.tight_layout()
plt.show()

ValueError: operands could not be broadcast together with shapes (270,231) (270,232) 